<a href="https://colab.research.google.com/github/bhargavigulla75/FMML/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

The size of the validation set plays a crucial role in assessing the performance of a machine learning model. It can have significant implications for model training and evaluation. Let's consider the impact of increasing and decreasing the percentage of the validation set:

Increasing the Percentage of the Validation Set:

Pros:

More Data for Evaluation: With a larger validation set, you have more data for evaluating your model's performance. This can lead to more reliable and stable estimates of your model's performance because you're testing it on a larger portion of unseen data.
Better Generalization Assessment: A larger validation set can provide a better indication of how well your model will generalize to new, unseen data.
Cons:

Less Data for Training: As you allocate a larger percentage of your data to the validation set, you have less data available for training. This can lead to a model that is less robust and might not perform as well on the training data.

2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

The sizes of the training and validation sets can significantly impact how well you can predict the accuracy on the test set using the validation set. The relationship between these sets is crucial in estimating a model's generalization performance. Here's how the sizes of the training and validation sets can affect your ability to predict test set accuracy:

Large Training Set:

Pros:

When you allocate a large portion of your data to the training set, the model has more data to learn from. This can result in a more robust and well-generalizing model.
A model trained on a large training set may capture more of the underlying patterns in the data, which can lead to better generalization.
Cons:

A smaller validation set means you have fewer data points to assess the model's performance. The estimate of the model's generalization ability may be less precise and more sensitive to the specific data points in the validation set.
There's a higher risk of overfitting the model to the training data if the validation set is significantly smaller. This may result in an overconfident but poorly generalizing model.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

The percentage of data reserved for the validation set is a crucial hyperparameter to determine when building a machine learning model. There's no one-size-fits-all answer, as the ideal split ratio between the training set and the validation set depends on various factors, including the size of your dataset, the complexity of your model, and the nature of your problem. It's a trade-off between having enough data for training and having a reliable estimate of your model's performance. Here are some general guidelines to help you find a balanced split:

Common Split Ratios:

A common starting point is an 80-20 or 70-30 split, with 80% (or 70%) of your data allocated to the training set and 20% (or 30%) to the validation set. This provides a reasonable balance between training data and validation data.
Use Cross-Validation:

Consider using cross-validation techniques, such as k-fold cross-validation. Cross-validation helps in utilizing your data more efficiently, providing more robust

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [14]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [15]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits using techniques like k-fold cross-validation can provide more consistent and reliable results compared to a single train-validation split. Cross-validation helps to reduce the variability in model performance estimates and provides a more robust evaluation of your machine learning model. Here's why averaging across multiple splits is beneficial:

Reduced Variability: Cross-validation involves partitioning your dataset into multiple subsets (folds) and training and evaluating the model multiple times. By averaging the performance over these iterations, you reduce the impact of a single, random partition that might lead to overly optimistic or pessimistic performance estimates. This makes your performance estimate more stable and reliable.

Improved Generalization Assessment: Cross-validation provides a better assessment of how well your model will generalize to new, unseen data. By averaging the performance across multiple folds, you get a more comprehensive view of the model's ability to handle different subsets of the data.
2. Does it give more accurate estimate of test accuracy?

Cross-validation, when used to estimate the model's performance, provides a more accurate estimate of test accuracy compared to a single train-validation split. While cross-validation doesn't directly estimate the accuracy on a held-out test set, it offers a better approximation of how your model is likely to perform on unseen data. Here's why it provides a more accurate estimate of test accuracy:

Reduced Overfitting to a Single Validation Set: In a single train-validation split, the model may inadvertently overfit to the specific data in the validation set. This can result in overly optimistic or pessimistic performance estimates. Cross-validation, by averaging performance across multiple validation sets, mitigates this overfitting issue, leading to a more accurate representation of how the model will perform on unseen data.

Better Generalization Assessment: Cross-validation allows your model to be trained and evaluated on various subsets of the data.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

The number of iterations (or folds) in cross-validation can affect the accuracy of the performance estimate. In general, increasing the number of iterations can lead to a more stable and reliable estimate of a model's performance. However, there are trade-offs to consider:

Pros of More Iterations:

Increased Stability: With more iterations, you get a more stable and less variable estimate of the model's performance. The estimate becomes less sensitive to the specific data partition in each fold.

Better Generalization Assessment: A larger number of iterations allows your model to be tested on a more diverse set of data subsets, providing a more comprehensive evaluation of its generalization ability.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Increasing the number of iterations (folds) in cross-validation can help mitigate the challenges associated with having a very small training dataset or validation dataset to some extent, but it may not completely resolve all the issues. Here are some considerations:

Benefits of Increasing Iterations with a Small Training or Validation Dataset:

Maximizing Data Utilization: More iterations allow you to make better use of the limited data available for training and validation. By cycling through different subsets of data, you can extract more information and potentially build a more robust model.

Reduced Impact of Random Variability: A larger number of iterations can reduce the impact of random variability in the data partitioning process. This means that your performance estimate is less likely to be heavily influenced by a single, unlucky data split.
